# 📊 BEDROT PRODUCTIONS — Streaming Revenue & Profitability Analysis  
*Data window: Oct-2024 → May-2025*  

<small>Notebook generated <<date>>.</small>

**Table of Contents**

1. [Imports & Config](#1)  
2. [Load Raw Data](#2)  
3. [Monthly Revenue Consolidation](#3)  
4. [MoM Growth Calculations](#4)  
5. [Forecast Models](#5)  
    * 5.1 Holt-Winters (Multiplicative)  
    * 5.2 Log-ARIMA (Drift)  
6. [Scenario Builder (Best | Base | Worst)](#6)  
7. [Cost Architecture](#7)  
8. [Profitability Timeline Simulation](#8)  
9. [Dashboards & Business KPIs](#9)  
10. [Appendix — Helper Utilities](#10)


In [3]:
# %% [markdown]
# # 🛠 0 Environment Setup
# This cell auto-installs any missing libraries so the notebook runs end-to-end
# on a fresh Jupyter/Lab kernel. Nothing else in the workflow changes.

# %%
import importlib, subprocess, sys, warnings
warnings.filterwarnings("ignore")

def _ensure(pkg:str):
    """Import `pkg`; if missing, install via pip silently."""
    try:
        importlib.import_module(pkg)
    except ModuleNotFoundError:
        print(f"↳ Installing missing dependency: {pkg} …")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
        print(f"   ✔ {pkg} installed.")

for _p in ["statsmodels", "pandas", "numpy", "matplotlib", "scikit-learn"]:
    _ensure(_p)

print("✅  All dependencies satisfied.")


↳ Installing missing dependency: statsmodels …
   ✔ statsmodels installed.
↳ Installing missing dependency: scikit-learn …
   ✔ scikit-learn installed.
✅  All dependencies satisfied.


In [5]:
# %% [markdown]
# ## 1 Imports & global settings  
# We load scientific, plotting, and forecasting libraries.  
# Two environment variables hold the payout file paths so the notebook stays portable
# across machines (no hard-coded local paths).

# %
import os, math, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error

warnings.filterwarnings("ignore")
plt.style.use("default")
pd.options.display.float_format = "{:,.2f}".format

CAPITAL_ONE_CSV = Path(os.getenv("CAPITAL_ONE_CSV", "MISSING_CAPITAL_ONE.csv"))
DISTROKID_TSV  = Path(os.getenv("DISTROKID_TSV",  "MISSING_DISTROKID.tsv"))
